In [5]:
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer
from datasets import load_dataset
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
import torch
import evaluate
import numpy as np

/Users/priyal/Documents/honours/mistral_env2/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
device = "mps" if torch.backends.mps.is_available() else "cpu"
print(f"Using device: {device}")

Using device: mps


In [7]:
# Load base model
model_name = "openai-community/gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token  # Set padding token

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float32,
    device_map={"": device} if device == "mps" else "auto",
)

# Enable gradient checkpointing for memory efficiency
model.gradient_checkpointing_enable()
model.config.use_cache = False  # Disable KV cache for training

In [4]:
dataset_path = "/Users/priyal/Documents/honours/datasets/qa_dataset_cleaned.csv"
dataset = load_dataset("csv", data_files=dataset_path)

# Split dataset 90/10 for training and evaluation
splits = dataset["train"].train_test_split(test_size=0.1, seed=42)
train_dataset = splits["train"]
eval_dataset = splits["test"]

print(f"Training examples: {len(train_dataset)}")
print(f"Evaluation examples: {len(eval_dataset)}")

Training examples: 518
Evaluation examples: 58


In [2]:
# Improved tokenization function with padding and attention masks
def tokenize_function(examples):
    # Format text as instruction-input-output
    formatted_texts = [
        f"Instruction: {i}\nInput: {inp}\nOutput: {outp}"
        for i, inp, outp in zip(
            examples["instruction"], examples["input"], examples["output"]
        )
    ]

    # Tokenize with proper padding
    result = tokenizer(
        formatted_texts,
        padding="max_length",
        truncation=True,
        max_length=512,  # Increased max length to fit longer examples
        return_tensors=None,  # Return Python lists
    )

    # Create labels for causal LM (shift inputs right)
    result["labels"] = result["input_ids"].copy()

    # Set -100 for padding tokens to ignore them in loss calculation
    for i, label_seq in enumerate(result["labels"]):
        # Find where the attention mask is 0 (padding tokens)
        pad_positions = [j for j, v in enumerate(result["attention_mask"][i]) if v == 0]
        for pos in pad_positions:
            result["labels"][i][pos] = -100

    return result

In [6]:
# Tokenize datasets
tokenized_train = train_dataset.map(
    tokenize_function, batched=True, remove_columns=train_dataset.column_names
)

tokenized_eval = eval_dataset.map(
    tokenize_function, batched=True, remove_columns=eval_dataset.column_names
)

Map: 100%|██████████| 58/58 [00:00<00:00, 1193.59 examples/s]


In [7]:
# Apply LoRA configuration with more targets
peft_config = LoraConfig(
    task_type="CAUSAL_LM",
    r=16,  # Increased rank
    lora_alpha=32,
    lora_dropout=0.1,
    # Target more modules for better adaptation
    target_modules=["c_attn", "c_proj", "c_fc"],
    bias="none",
    inference_mode=False,
)

In [8]:
# Apply LoRA to model
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

'NoneType' object has no attribute 'cadam32bit_grad_fp32'
trainable params: 2,359,296 || all params: 126,799,104 || trainable%: 1.8607


/Users/priyal/Documents/honours/mistral_env2/lib/python3.11/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "
/Users/priyal/Documents/honours/mistral_env2/lib/python3.11/site-packages/peft/tuners/lora/layer.py:1264: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


In [9]:
# Training arguments with longer training and evaluation
training_args = TrainingArguments(
    output_dir="./results_improved",
    learning_rate=2e-5,
    lr_scheduler_type="cosine",
    warmup_ratio=0.1,
    per_device_train_batch_size=2,  # Increase if possible
    gradient_accumulation_steps=8,  # Effective batch size of 16
    num_train_epochs=10,  # Train for longer
    weight_decay=0.01,  # Add regularization
    logging_dir="./logs_improved",
    logging_steps=10,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=3,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    report_to="tensorboard",
    fp16=False,  # MPS doesn't support fp16
    dataloader_num_workers=4,
    remove_unused_columns=True,
    group_by_length=True,  # More efficient batching
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_eval,
    tokenizer=tokenizer,
)

/Users/priyal/Documents/honours/mistral_env2/lib/python3.11/site-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/var/folders/4y/9y5ld1xn3zn_rhkhdmmr63zr0000gn/T/ipykernel_64141/369941315.py:26: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [10]:
# Train the model
print("Starting fine-tuning")
train_result = trainer.train()
trainer.save_model("./gpt2_improved")
tokenizer.save_pretrained("./gpt2_improved")

# Print training metrics
print(f"Training metrics: {train_result.metrics}")
final_eval = trainer.evaluate()
print(f"Final evaluation metrics: {final_eval}")

Starting fine-tuning


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

Epoch,Training Loss,Validation Loss
1,3.358200,3.107535
2,3.281000,3.070785
3,3.222100,3.020990
4,3.145000,3.004219
5,3.096600,2.973629
6,3.127700,2.972182
7,3.114600,2.964339
8,3.119300,2.944131
9,3.117300,2.945190


Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can e

Training metrics: {'train_runtime': 8687.9755, 'train_samples_per_second': 0.596, 'train_steps_per_second': 0.037, 'total_flos': 1350755140239360.0, 'train_loss': 3.1710935056209566, 'epoch': 9.71042471042471}


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

Final evaluation metrics: {'eval_loss': 2.955711603164673, 'eval_runtime': 77.3387, 'eval_samples_per_second': 0.75, 'eval_steps_per_second': 0.103, 'epoch': 9.71042471042471}


In [9]:
def generate_response(instruction, input_text="", max_new_tokens=200):
    prompt = f"Instruction: {instruction}\nInput: {input_text}\nOutput: "

    inputs = tokenizer(prompt, return_tensors="pt").to(device)

    with torch.no_grad():
        output = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            temperature=0.7,
            top_p=0.9,
            top_k=50,
            num_beams=3,
            no_repeat_ngram_size=3,
            pad_token_id=tokenizer.eos_token_id,
        )

    full_output = tokenizer.decode(output[0], skip_special_tokens=True)

    if prompt in full_output:
        return full_output[len(prompt) :].strip()
    elif "Output: " in full_output:
        return full_output.split("Output: ")[1].strip()
    return full_output


instruction = "How can Ayurveda contribute to the management and treatment of epilepsy"
input_text = """Epilepsy, a chronic non-communicable disease of the brain, is one of the most common neurological diseases globally that affects people of all ages. The existence of medical, neurological, psychiatric, and cognitive comorbidities has always undermined the available advanced treatment strategies for epilepsy. New-generation antiepileptic drugs being less successful in completely controlling the seizures and observance of complex diseases, including drug-resistant cases, have provided scope for integrating and incorporating the therapeutic modalities of Ayurveda, the ancient Indian art of holistic medicine, in the effective management of epilepsy. Epilepsy can be correlated to Apasmara, described in the classics of Ayurveda as the transient appearance of unconsciousness with loathsome expression due to derangement of memory, intelligence, and mind. The multifaceted therapeutic approach of Ayurveda, which involves pharmacologic and nonpharmacologic measures, purificatory and pacifying procedures, herbal and herbo-mineral formulations, disease, and host-specific approaches, have enhanced the potential of not only relieving symptoms but also modifying the pathophysiology of the disease. Newer paradigms of research in Ayurveda, along with holistic and integrative approaches with contemporary medicine, can not only benefit the existing healthcare system but also impact future healthcare management in epileptology research. This cursory literature review is an earnest attempt to identify, evaluate, and summarize various studies and provide a comprehensive insight into the potential of Ayurveda in understanding and treating epilepsy."""

response = generate_response(instruction, input_text)
print("\nGenerated response:")
print(response)


Generated response:
______________________________________________________________________________
The following is a brief summary of the literature review.
1. Introduction
Epilepsy is a chronic neurodegenerative disorder characterized by progressive loss of consciousness, loss of function, and loss of the ability to perform basic functions of life. The symptoms of epilepsy are characterized by a loss of conscious awareness and a lack of control over the body. The disease is characterized by an inability to distinguish between the body and the mind, and the inability of the body to recognize and control the body's actions. The condition is associated with an increased risk of death and disability. It is also associated with a higher risk of mental retardation, and with a greater risk of developing dementia. In addition, the condition can lead to a variety of health problems, including:

Acute neuropsychiatric disorders

Seizures

Inflammatory bowel disease (IBS)

Chronic obstructive 

In [10]:
instruction = "Has the Indian government promoted Ayurveda?"
input_text = """ETHNOPHARMACOLOGICAL RELEVANCE Ayurveda Sanskrit Ayus - life Veda - knowledge means the True knowledge of life. Ayurveda deals with a complete self-sustainable system of medicine. The Government of India through its Ministry of AYUSH is responsible for policy formulation, development and implementation of programs for the growth, development and propagation of Ayurveda. AIM OF THE STUDY This review aimed to highlight the various aspects of government policies and initiatives for development of Ayurveda. MATERIALS AND METHODS We critically reviewed various books, annual reports, policy documents and various ancient Ayurvedic literatures. Besides the websites of Ministry of AYUSH, National Medicinal Plant Board, Central Council for Research on Ayurvedic Sciences CCRAS and AYUSH research portal have been searched and data was recorded. RESULTS The vision of the ministry is to position AYUSH systems as the preferred systems of living and practice for attaining healthy nation. The ministry has identified its mission in terms of seven broad thematic functional areas of AYUSH activities. These are information, education and communication drug administration, human resource development, medicinal plants, research and development, international collaborations, AYUSH services. Different programs have been taken up towards increasing visibility, acceptability and usage of Ayurveda vis-a vis its integration in the health system. Strategies to globalize and promote Ayurveda are being taken up through AYUSH clusters focusing its safety-efficacy-quality aspects and rational use of Ayurveda CONCLUSION The government policies are taking firm steps towards promotion and development of Ayurveda. Research and development towards validation of Ayurveda is being projected as the thrust area."""

response = generate_response(instruction, input_text)
print("\nGenerated response:")
print(response)


Generated response:
_________________________________________________________________________
The government has identified seven broad areas of its efforts to promote and develop Ayurvea. These include: (1) the promotion of the use of medicines, (2) the establishment of a national pharmacopoeia, (3) the creation of a National Medical Council, and (4) the development of the National Medical Research Council (MMRC). The government has also identified a number of other areas where it is working to promote the use and use of medicine in the country. The government is also working on the promotion and dissemination of medicines in the public health sector. In addition, the government is working on developing a national medical research council (MMRCC) to be established in the next five years. The Ministry of Health and the Ministry of Education have also been involved in promoting the promotion, dissemination, and promotion of medicines. The Department of Health has also been engaged in p

In [11]:
instruction = "What can I use Ashwagandha for?"
input_text = """ETHNOPHARMACOLOGICAL RELEVANCE Withania somnifera L. Dunal Physalis somnifera L. is a fairly known perennial shrub of Solanaceae family, and is used in Ayurveda- Traditional Indian Medicine TIM, since ancient times. It is well known as Ashwagandha in Sanskrit language in Ayurvedic classics. Its Mula root is recommended for health and healing, and the number of single and compound formulation is prescribed rationally. It is believed that the species name-somnifera is coined based on popular use to induce sleep in Ayurveda. AIM OF THE STUDY The present study was aimed to bring out the experience-based traditional uses of Ashwagandha for health and healing with an emphasis on the pharmacological and biochemical scientific evidences to corroborate them. The scientific evidences have been explored from the national and international publications. MATERIALS AND METHOD A comprehensive literary search of Ayurvedic classics was carried out systematically regarding Ashwagandha for its rationality behind the traditional uses. To excavate the subject matter, the original Ayurvedic scriptures and several standard Ayurvedic texts of different period was studied insightfully for meaningful contribution. It is to be noted that the primary source of knowledge was considered in writing this manuscript without any biased attitude. The available literature on Ashwagandha was also searched to ascertain the basis of scientific Latin name and correct identity. The Ayurvedic Pharmacopoeia and other relevant scientific works were also taken into consideration to make the subject matter more clearly to the scientific world. For the scientific evidence of the uses, the international and national Journals and other published material were also searched to make it inquisitiveness to the scholars interested in Ayurvedic medicinal plants. RESULTS The present paper throws ancient luminosity behind the therapeutic uses of one of the promising plant drug i.e., Ashwagandha of ancient India even to the present time. The scientific evidences corroborate the rationality ascribed in available Ayurvedic classics of various period of India has been gained. CONCLUSION The study explores that the first reference of Ashwagandha with its significant nomenclature, useful part, properties, action, and eloquent uses has its footprint in the original texts of Ayurveda. In later works enhanced knowledge with traditional uses continued even today. Several single and compound formulations have been found to maintain the health and to alleviate the disorders rationally. It is worthy to note here that the scientific evidences corroborate the uses practiced in Ayurveda."""
response = generate_response(instruction, input_text)
print("\nGenerated response:")
print(response)


Generated response:
______________________________________________________________________________
The present study shows that the ancient medicinal plants of Ashwegandha have been used to treat diseases of the body and mind. It also shows that Ashwegatha has been used as a therapeutic medicine for the treatment of various diseases. It shows that it is a medicinal plant for the prevention of diseases. The results of the present study show that the medicinal plants are useful for the protection of the health of the patient.
The results of this study are summarized in Table 1.
TABLE 1
TABLE 2
TABLE 3
TABLE 4
TABLE 5
TABLE 6
TABLE 7
TABLE 8
TABLE 9
TABLE 10
TABLE 11
TABLE 12
TABLE 13
TABLE 14
TABLE 15
TABLE 16
TABLE 17
TABLE 18
TABLE 19
TABLE 20
TABLE 21
TABLE 22
TABLE 23
TABLE 24
TABLE 25
TABLE 26
TABLE 27
TABLE 28
TABLE 29
TABLE 30
TABLE 31
TABLE 32
TABLE


In [12]:
instruction = "What Ayurvedic herb can I take to reduce hyperacidity?"
input_text = """AIM The aim of this article is to review the current literature on the therapeutic uses and efficacy of Triphala. Herbal remedies are among the most ancient medicines used in traditional systems of healthcare such as Ayurveda. Triphala, a well-recognized and highly efficacious polyherbal Ayurvedic medicine consisting of fruits of the plant species Emblica officinalis Amalaki, Terminalia bellerica Bibhitaki, and Terminalia chebula Haritaki, is a cornerstone of gastrointestinal and rejuvenative treatment. METHODS A search of the PubMed database was conducted. RESULTS In addition, numerous additional therapeutic uses described both in the Ayurvedic medical literature and anecdotally are being validated scientifically. In addition to laxative action, Triphala research has found the formula to be potentially effective for several clinical uses such as appetite stimulation, reduction of hyperacidity, antioxidant, anti-inflammatory, immunomodulating, antibacterial, antimutagenic, adaptogenic, hypoglycemic, antineoplastic, chemoprotective, and radioprotective effects, and prevention of dental caries. Polyphenols in Triphala modulate the human gut microbiome and thereby promote the growth of beneficial Bifidobacteria and Lactobacillus while inhibiting the growth of undesirable gut microbes. The bioactivity of Triphala is elicited by gut microbiota to generate a variety of anti-inflammatory compounds. CONCLUSIONS This review summarizes recent data on pharmacological properties and clinical effects of Triphala while highlighting areas in need of additional investigation and clinical development."""
response = generate_response(instruction, input_text)
print("\nGenerated response:")
print(response)


Generated response:
_________________________________________________________________________
In this article, we review the literature on Triphal and its potential therapeutic use in the treatment of chronic inflammatory bowel disease (CID). We also discuss the potential therapeutic applications of triphala in the management of CID. In this review, we summarize the current data on the pharmacological effects and potential therapeutic uses of TriPhala. We also review the potential clinical applications of TriPHAL in the prevention and treatment of inflammatory bowel diseases (IBD). In this section, we discuss the current evidence on the efficacy and safety of Tri PHAL in IBD.
In the present review, the aim of the present study was to evaluate the efficacy, safety, and tolerability of Tri phalate in the control of IBD in patients with Crohn's disease. The objective of this study was the development of a systematic review and meta-analysis of randomized controlled trials (RCTs) of Tri P